# init

In [ ]:
import sys, os, math
import numpy as np
import pandas as pd

from shapely.geometry import Point, LineString, MultiLineString
import shapely.geometry as sg

from matplotlib import pyplot as plt, axes
from timeit import default_timer as timer
from datetime import timedelta
from pprint import pprint

from libs.pyDrawNetworklib import plot_regions, plot_triangulation, plot_norm

from flat_norm_fixture import FlatNormFixture, close_fig, get_fig_from_ax
MIN_X, MIN_Y, MAX_X, MAX_Y = 0, 1, 2, 3
long_dashed = (5, (10, 3))

city_info = {
    'mcbryde': {
        'flatnorm': {
              1000: 0.438543,
             25750: 0.520787,
             50500: 0.582910,
             75250: 0.630447,
            100000: 0.669575,
        },
        'ratio': 0.52792,
        'length': 0.01071,
        'epsilon': 0.020293
    }
}
fx = FlatNormFixture('runTest')
fx.fig_dir = "figs/notebooks"
fx.out_dir = "outs/notebooks"
fx.area = 'mcbryde'


# Sample and Plot regions

## sample_regions()

In [ ]:
area = 'mcbryde'
epsilon = 2e-3
num_regions = 6
region = 2

# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

# sample regions
point_list, region_list = fx.sample_regions(
    hull, num_regions=num_regions, epsilon=epsilon, seed=54321
)

# plot regions
fig, ax = fx.plot_regions_list(
    act_geom, synt_geom, region_list, area,
    show=True,
    region_highlight=region,
    file_name_sfx=f"sr_region_{region}",
    figsize=(40, 20),
    do_return=True
)

## sample_regions_geom()

In [ ]:
area = 'mcbryde'
epsilon = 2e-3
num_regions = 6
region = 2


# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

# sample regions
point_list, region_list = fx.sample_regions_geom(
    hull, act_geom, synt_geom,
    num_regions=num_regions, epsilon=epsilon,
    seed=54321
)

# plot regions
fig, ax = fx.plot_regions_list(
    act_geom, synt_geom, region_list, area,
    show=True,
    region_highlight=region,
    file_name_sfx=f"srgeom_region_{region}",
    figsize=(40, 20),
    do_return=True
)

## plot city region

In [ ]:
area = 'mcbryde'
epsilon = 2e-3

# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

city_region = sg.box(*hull.exterior.bounds)

# plot regions
fig, ax = fx.plot_regions_list(
    act_geom, synt_geom, [city_region], area,
    file_name_sfx=f"city",
    figsize=(40, 20),
    do_return=True,
    show=True,
)

In [ ]:
p = Point(0, 0).buffer(1, cap_style=sg.base.CAP_STYLE.square)
list(p.exterior.coords), type(p)

In [ ]:
hull.bounds

In [ ]:
city_bounds = hull.exterior.bounds
city_bounds[MAX_X] - city_bounds[MIN_X]

In [ ]:
list(sg.box(*hull.exterior.bounds).exterior.coords)

In [ ]:
sg.box(*hull.exterior.bounds)
sg.box(*hull.exterior.bounds)


# Compute and Plot Flatnorm

## region flatnorm

In [ ]:
area = 'mcbryde'
epsilon, lambda_ = 2e-3, 1000
num_regions = 6
region = 2
fx.seed=54321

# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

# sample regions
point_list, region_list = fx.sample_regions_geom(
    hull, act_geom, synt_geom,
    num_regions=num_regions, epsilon=epsilon,
    seed=54321
)

# plot regions
fx.plot_regions_list(
    act_geom, synt_geom, region_list, area,
    region_highlight=region,
    file_name_sfx=f"srg_region_{region}",
    do_return=False,
    figsize=(40, 20)
)

# flat norm
norm, enorm, tnorm, w, plot_data = fx.compute_region_flatnorm(
    region_list[region], act_geom, synt_geom,
    lambda_=lambda_,
    normalized=True,
    plot=True
)

# plot flat norm
fx.plot_triangulated_region_flatnorm(
    epsilon=epsilon, lambda_=lambda_,
    to_file=f"{area}-flatnorm_region_{region}",
    suptitle_sfx=f"FN={norm:0.5f} : |T| = {w:0.5f} : |T| / $\\epsilon$ = {w / epsilon:0.5f}",
    do_return=False, show=True,
    **plot_data
)


## city flatnorm

In [ ]:
fx.area = 'mcbryde'
lambda_ = 1000

# read geometries
act_geom, synt_geom, hull = fx.read_networks()

# city region
city_bounds = hull.exterior.bounds
city_region = sg.box(*city_bounds)
city_width, city_height = city_bounds[MAX_X] - city_bounds[MIN_X], city_bounds[MAX_Y] - city_bounds[MIN_Y]
epsilon = max(city_width/2, city_height/2)

# plot city region
fx.plot_regions_list(
    act_geom, synt_geom, [city_region], fx.area,
    file_name_sfx=f"city",
    do_return=False, show=True,
    figsize=(40, 20)
)

# city flat norm
norm, enorm, tnorm, w, plot_data = fx.compute_region_flatnorm(
    city_region, act_geom, synt_geom,
    lambda_=lambda_,
    normalized=True,
    plot=True
)

# plot flat norm
fx.plot_triangulated_region_flatnorm(
    epsilon=f"{epsilon:0.4f}", lambda_=lambda_,
    to_file=f"{area}-flatnorm_city",
    suptitle_sfx=f"$F_{{\\lambda}}$={norm:0.5f} : "
                 f"|T| = {w:0.5f} : "
                 f"|T| / $\\epsilon$ = {w / epsilon:0.5f}",
    do_return=False, show=True, constrained_layout=True,
    **plot_data
)


# Plot Flatnorm lines

## region flatnorm lines

In [ ]:
area = 'mcbryde'
epsilons, lambdas = np.linspace(5e-4, 2e-3, 4), np.linspace(1000, 100000, 5)
num_regions = 6
region = 2
fx.seed=54321

# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

points, regions = fx.sample_regions_geom(
    hull, act_geom, synt_geom,
    epsilon=epsilons[0],
    num_regions=num_regions,
    seed=fx.seed
)

# plot regions
fx.plot_regions_list(
    act_geom, synt_geom, regions, area,
    region_highlight=region,
    file_name_sfx=f"_region_{region}",
    do_return=False, show=True,
    figsize=(40, 20)
)

# flat norm
flatnorm_data = {
    'epsilons': [], 'lambdas': [], 'flatnorms': [],
    'norm_lengths': [], 'norm_areas': []
}

start = timer()
for epsilon in epsilons:
    for lambda_ in lambdas:
        norm, enorm, tnorm, w = fx.compute_region_flatnorm(
            fx.get_region(points[region], epsilon),
            act_geom, synt_geom,
            lambda_=lambda_,
            normalized=True,
            plot=False
        )
        flatnorm_data['epsilons'].append(f"{epsilon:0.4f}")
        flatnorm_data['lambdas'].append(lambda_)
        flatnorm_data['flatnorms'].append(norm)
        flatnorm_data['norm_lengths'].append(enorm)
        flatnorm_data['norm_areas'].append(tnorm)

end = timer()

fig, ax = fx.plot_region_flatnorm_lines(
    epsilons=flatnorm_data['epsilons'],
    lambdas=flatnorm_data['lambdas'],
    flatnorms=flatnorm_data['flatnorms'],
    to_file=f"{area}-flatnorm-lines_region_{region}",
    do_return=True
)

print("--------------------------------------------------------------------------")
print(f"compute region flatnorm "
      f"for {len(epsilons)} epsilons "
      f"and {len(lambdas)} lambdas = {timedelta(seconds=end-start)}")
print("--------------------------------------------------------------------------")
flatnorm_data = pd.DataFrame(flatnorm_data)
flatnorm_data.head()


## city flatnorm lines

# flatnorm stats

In [ ]:
fx.out_dir = "out/test"
%ls {fx.out_dir}

## load regions stats

In [ ]:
fx.area = 'mcbryde'
epsilons, lambdas = np.linspace(5e-4, 2e-3, 4), np.linspace(1000, 100000, 5, dtype=int)
num_regions = 50
fx.out_dir = "out/test"
file_name = f"{fx.area}-flatnorm-stats_{num_regions}_regions"

flatnorm_df = pd.read_csv(
    f"{fx.out_dir}/{file_name}.csv",
    sep=",",
    dtype={
         'epsilons': float,
         'lambdas': int,
         'flatnorms': np.float64,
         'norm_lengths': np.float64,
         'norm_areas': np.float64,
         'input_lengths': np.float64,
         'input_ratios': np.float64,
         'MIN_X': np.float64,
         'MIN_Y': np.float64,
         'MAX_X': np.float64,
         'MAX_Y': np.float64,
    }
)
flatnorm_df['id'] = list(range(len(flatnorm_df)))
flatnorm_df = flatnorm_df.set_index(['epsilons', 'lambdas'], drop=False)
# flatnorm_df.head(10)

## load city stats

In [ ]:
fx.area = 'mcbryde'
fx.out_dir = "out/test"
file_name = f"{fx.area}-flatnorm-stats_city"

fn_city_df = pd.read_csv(
    f"{fx.out_dir}/{file_name}.csv",
    sep=",",
    # header=
)
fn_city_df = fn_city_df.set_index(['lambdas'], drop=False)
city_ratio = fn_city_df['input_ratios'].max()
fn_city_df

## histograms

### plot_hist_fn()

In [ ]:
def plot_hist_fn(fn_df, city_df, lambda_, epsilon=None, ax=None, **kwargs):
    fig, ax, _ = get_fig_from_ax(ax, **kwargs)
    lambda_ = int(lambda_)

    colors = {
        'city': kwargs.get('city_color', 'xkcd:electric blue'),
        'hist': kwargs.get('hist_color', 'xkcd:pastel blue'),
        'fn_mean': kwargs.get('fn_mean_color', 'xkcd:kelly green'),
    }


    # flatnorm VS ratios ------------------------------------------------------
    if epsilon:
        fn_df = fn_df.loc[(epsilon, lambda_), ].copy()
    else:
        idx = pd.IndexSlice
        fn_df = fn_df.loc[idx[:, lambda_], ].copy()

    # flatnorm histogram -------------------------------------------------------
    hist_data = ax.hist(
        fn_df['flatnorms'],
        bins=20, color=colors['hist'],
        range=(0, 1),
    )


    # mean flatnorm -----------------------------------------------------------
    fn_mean = fn_df['flatnorms'].mean()
    fn_sd = fn_df['flatnorms'].std()
    ax.plot(
        [fn_mean, fn_mean], [0, 100],
        c=colors['fn_mean'], linewidth=3
    )
    ax.plot(
        [fn_mean - fn_sd, fn_mean - fn_sd], [0, 100],
        c=colors['fn_mean'], linewidth=1, linestyle=long_dashed
    )
    ax.plot(
        [fn_mean + fn_sd, fn_mean + fn_sd], [0, 100],
        c=colors['fn_mean'], linewidth=1, linestyle=long_dashed
    )

    # city flatnorm -----------------------------------------------------------
    city_fn = city_df.at[lambda_, 'flatnorms']
    ax.plot(
        [city_fn, city_fn] , [0, 100],
        c=colors['city'], linewidth=3
    )

    # ticks -------------------------------------------------------------------
    ax.set_xticks([city_fn, fn_mean],
                  labels=[f"{city_fn:0.3f}", f"{fn_mean:0.3f}"],
                  minor=True,
                  )

    for tlabel, tcolor in zip(ax.get_xticklabels(minor=True), [colors['city'], colors['fn_mean']]):
        tlabel.set_color(tcolor)

    # plot title --------------------------------------------------------------
    titles = dict(
        fn = f"$\\hat{{F}}_{{\\lambda}}={fn_mean:0.3f} \pm {fn_sd:0.3f}$",
        city = f"$F_{{\\lambda}}^{{C}} = {city_fn:0.4f}$"
    )
    which_titles = kwargs.get('titles', list(titles.keys()))

    title_styles = dict(
        ll=f"$\\lambda={{lambda_:d}}$",
        le=f"$\\lambda={{lambda_:d}}$, $\\epsilon={{epsilon:0.4f}}$",
        el=f"$\\epsilon={{epsilon:0.4f}}$, $\\lambda={{lambda_:d}}$",
    )
    title_style = kwargs.get('title_style', 'le')

    if epsilon:
        title = title_styles[title_style].format(lambda_=lambda_, epsilon=epsilon)
    else:
        title = title_styles['ll'].format(lambda_=lambda_)

    # subtitle = ", ".join([t_str for t_name, t_str in titles.items() if t_name in which_titles])
    subtitle = ", ".join([titles[t_name] for t_name in which_titles])
    title = f"{title}\n{subtitle}" if subtitle else title
    ax.set_title(title, fontsize=kwargs.get('title_fontsize', 18))

    # plot config -------------------------------------------------------------
    ax.set_xlabel("$F_{{\\lambda}}$",
                  # rotation='horizontal',
                  fontsize=kwargs.get('xylabel_fontsize', 16))

    ax.set_ylim(bottom=0, top=hist_data[0].max() + 1)
    ax.set_xlim(left=-0.02, right=1.02)
    return fn_mean, city_fn



### by lambdas

In [ ]:
# BY LAMBDAS
L = len(lambdas)
idx = pd.IndexSlice
fig, axs = plt.subplots(1, L, figsize=(L*10, 10), constrained_layout=True)

fn_means, fn_cities = list(), list()
for l, lambda_ in enumerate(lambdas):
    fnm, fnc = plot_hist_fn(
        flatnorm_df, fn_city_df, lambda_, ax=axs[l],
        titles=['fn', 'city'],

    )
    fn_means.append(fnm)
    fn_cities.append(fnc)
    pass

fnm_mean, fnm_std = np.array(fn_means).mean(), np.array(fn_means).std()
fnc_mean, fnc_std = np.array(fn_cities).mean(), np.array(fn_cities).std()

# fnm_suptitle = f"mean($\\hat{{F}}_{{\\lambda}}$)={fnm_mean:0.3f}, sd($\\hat{{F}}_{{\\lambda}}$)={fnm_std:0.3f}"
fnm_suptitle = f"${{\\sf mean}}(\\hat{{F}}_{{\\lambda}})={fnm_mean:0.3f}, {{\\sf sd}}(\\hat{{F}}_{{\\lambda}})={fnm_std:0.3f}$"
fnm_suptitle_short = f"$\\hat{{F}}_{{\\lambda}}={fnm_mean:0.3f} \\pm {fnm_std:0.3f}$"
city_suptitle = f"${{\\bf {fx.area}}} : F_{{\\lambda}}^{{C}}={fnc_mean:0.3f} \\pm {fnc_std:0.3f}$"

fig.suptitle(f"{city_suptitle}\n{fnm_suptitle}", fontsize=25)

file_name = f"{fx.area}-flatnorm-hists-lambdas_{num_regions}_regions"
close_fig(fig, to_file=f"{fx.fig_dir}/{file_name}.png", show=True)


### by lambdas * epsilons

In [ ]:
L, E = len(lambdas), len(epsilons)
fig, axs = plt.subplots(E, L, figsize=(L*10, E*10), constrained_layout=True)

fn_means, fn_cities = list(), list()
for e, epsilon in enumerate(epsilons):
    for l, lambda_ in enumerate(lambdas):
        fnm, fnc = plot_hist_fn(
            flatnorm_df, fn_city_df, lambda_, epsilon=epsilon,
            ax=axs[e][l],
            titles=['fn', 'city'],
            title_style="el",
        )
        fn_means.append(fnm)
        fn_cities.append(fnc)
        pass

fnm_mean, fnm_std = np.array(fn_means).mean(), np.array(fn_means).std()
fnc_mean, fnc_std = np.array(fn_cities).mean(), np.array(fn_cities).std()

fnm_suptitle_short = f"$\\hat{{F}}_{{\\lambda}}={fnm_mean:0.3f} \\pm {fnm_std:0.3f}$"
city_suptitle = f"${{\\bf {fx.area}}} : F_{{\\lambda}}^{{C}}={fnc_mean:0.3f} \\pm {fnc_std:0.3f}$"

fig.suptitle(f"{city_suptitle}\n{fnm_suptitle}", fontsize=25)

file_name = f"{fx.area}-flatnorm-hists_{num_regions}_regions"
close_fig(fig, to_file=f"{fx.fig_dir}/{file_name}.png", show=True)


## scatter plots

### plot_fn_vs_ratio()

In [ ]:
def plot_fn_vs_ratio(fn_df, city_df, lambda_, epsilon=None, ax=None, **kwargs):
    """
    :param kwargs: highlight: set, titles: list, title_style: str, X_color: str, X_fontsize: int;
                   if ax is None: figsize, constrained_layout
    """
    fig, ax, _ = get_fig_from_ax(ax, **kwargs)
    lambda_ = int(lambda_)

    colors = {
        'city': kwargs.get('city_color', 'xkcd:electric blue'),
        'scatter': kwargs.get('scatter_color', 'xkcd:pastel blue'),
        'highlight': kwargs.get('highlight_color', 'xkcd:pumpkin'),
        'fn_mean': kwargs.get('fn_mean_color', 'xkcd:kelly green'),
        'regression': kwargs.get('regression_color', 'red'),
    }


    # flatnorm VS ratios ------------------------------------------------------
    if epsilon:
        fn_df = fn_df.loc[(epsilon, lambda_), ].copy()
    else:
        idx = pd.IndexSlice
        fn_df = fn_df.loc[idx[:, lambda_], ].copy()

    x_ratios = fn_df['input_ratios'].to_numpy()
    y_flatnorms = fn_df['flatnorms'].to_numpy()

    if highlight := kwargs.get('highlight', list()):
        if not isinstance(highlight, (list, set)):
            highlight = set(highlight)
        fn_df['color'] = colors['scatter']
        fn_df.loc[fn_df['id'].isin(highlight), 'color'] = colors['highlight']
        colors['scatter'] = fn_df['color'].to_list()

    ax.scatter(
        x_ratios, y_flatnorms,
        alpha=0.7, s=5 ** 2, c=colors['scatter'],
        marker='o'
    )

    if highlight:
        ax.scatter(
            fn_df.loc[fn_df['id'].isin(highlight), 'input_ratios'],
            fn_df.loc[fn_df['id'].isin(highlight), 'flatnorms'],
            alpha=0.8, s=12 ** 2, c=colors['highlight'],
            marker='x'
        )


    # mean line ---------------------------------------------------------------
    fn_mean = y_flatnorms.mean()
    fn_sd = y_flatnorms.std()
    ax.plot(
        [0, 100], [fn_mean, fn_mean],
        c=colors['fn_mean'], linewidth=3, alpha=0.5
    )
    ax.plot(
        [0, 100], [fn_mean - fn_sd, fn_mean - fn_sd],
        c=colors['fn_mean'], linewidth=1, linestyle=long_dashed, alpha=0.5,
    )
    ax.plot(
        [0, 100], [fn_mean + fn_sd, fn_mean + fn_sd],
        c=colors['fn_mean'], linewidth=1, linestyle=long_dashed, alpha=0.5,
    )

    # city flatnorm -----------------------------------------------------------
    city_fn, city_ratio = city_df.loc[lambda_, ['flatnorms', 'input_ratios']]
    ax.scatter(
        [city_ratio], [city_fn],
        alpha=1, s=15 ** 2, color=colors['city'],
        marker='*'
    )

    # regression line ---------------------------------------------------------
    from sklearn.metrics import r2_score
    # regression
    N = len(x_ratios)
    b, a = np.polyfit(x_ratios, y_flatnorms, deg=1)
    y_predict = a + b * x_ratios
    y_var = sum((y_predict - y_flatnorms) ** 2) / (N - 1)
    y_err = math.sqrt(y_var)
    b_var = y_var / sum((x_ratios - x_ratios.mean())**2)
    b_err = math.sqrt(b_var)
    # correlation & R2
    fn_ratio_corr = np.corrcoef(x_ratios, y_flatnorms)[0, 1]
    fn_ratio_r2 = r2_score(y_flatnorms, y_predict)
    xseq = np.linspace(0, 1, num=10)
    ax.plot(xseq, a + b * xseq, color=colors['regression'], lw=3, alpha=0.5)
    ax.plot(xseq, a + y_err + b * xseq, color=colors['regression'], lw=1, alpha=0.5, linestyle=long_dashed)
    ax.plot(xseq, a - y_err + b * xseq, color=colors['regression'], lw=1, alpha=0.5, linestyle=long_dashed)

    # stat data dict ----------------------------------------------------------
    stat_dict = {
        'a': a, 'b': b,
        'err': math.sqrt(y_var),
        'std_err': np.std(y_predict - y_flatnorms),
        'b_err': b_err,
        'corr': fn_ratio_corr,
        'R2': fn_ratio_r2,
        'fn_mean': fn_mean,
        'fn_sd': fn_sd,
    }

    # ticks -------------------------------------------------------------------
    ax.set_xticks([city_ratio], [f"{city_ratio:0.3f}"], color=colors['city'], minor=True, )
    ax.set_yticks([city_fn, fn_mean],
                  labels=[f"{city_fn:0.3f}", f"{fn_mean:0.3f}"],
                  minor=True,
                  # color=[colors['city'], colors['fn_mean']],
                  )

    for tlabel, tcolor in zip(ax.get_yticklabels(minor=True), [colors['city'], colors['fn_mean']]):
        tlabel.set_color(tcolor)

    # plot title --------------------------------------------------------------
    titles = dict(
        fn = f"$\\hat{{F}}_{{\\lambda}}={fn_mean:0.3f} \pm {fn_sd:0.3f}$",
        beta = f"$\\hat{{\\beta}}={b:0.3f} \pm {b_err:0.3f}$",
        corr = f"$\\rho={fn_ratio_corr:0.3f}$",
        r2 = f"$R^{{2}}={fn_ratio_r2:0.3f}$",
    )
    which_titles = kwargs.get('titles', list(titles.keys()))

    title_styles = dict(
        ll=f"$\\lambda={{lambda_:d}}$",
        le=f"$\\lambda={{lambda_:d}}$, $\\epsilon={{epsilon:0.4f}}$",
        el=f"$\\epsilon={{epsilon:0.4f}}$, $\\lambda={{lambda_:d}}$",
    )
    title_style = kwargs.get('title_style', 'le')

    if epsilon:
        title = title_styles[title_style].format(lambda_=lambda_, epsilon=epsilon)
    else:
        title = title_styles['ll'].format(lambda_=lambda_)


    # subtitle = ", ".join([t_str for t_name, t_str in titles.items() if t_name in which_titles])
    subtitle = ", ".join([titles[t_name] for t_name in which_titles])
    title = f"{title}\n{subtitle}"
    ax.set_title(title, fontsize=kwargs.get('title_fontsize', 18))

    # plot config -------------------------------------------------------------
    ax.set_xlabel("|T|/$\epsilon$", fontsize=kwargs.get('xylabel_fontsize', 16))
    ax.set_ylabel("$F_{{\\lambda}}$",
                  rotation='horizontal',
                  fontsize=kwargs.get('xylabel_fontsize', 16))
    ax.set_xlim(left=0, right=1)
    ax.set_ylim(bottom=0, top=1.05)
    return stat_dict


### by lambdas

In [ ]:
# BY LAMBDAS
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)
fig, axs = plt.subplots(1, L, figsize=(L*10, 11), constrained_layout=True)
idx = pd.IndexSlice
reg_coefs = []
reg_coefs_dict = dict()

for l, lambda_ in enumerate(lambdas):
    reg_coefs_dict[lambda_] = plot_fn_vs_ratio(
        flatnorm_df, fn_city_df, lambda_,
        ax=axs[l],
        titles=['fn', 'beta', 'r2'],
    )

reg_coefs_data = pd.DataFrame(reg_coefs_dict).transpose()
beta_mean, beta_std = reg_coefs_data['b'].mean(), reg_coefs_data['b'].std()

beta_suptitle = f"mean($\\hat{{\\beta}}$)={beta_mean:0.4f}, sd($\\hat{{\\beta}}$)={beta_std:0.4f}"
city_suptitle = f"${{\\bf {fx.area}}} : |T|/\\epsilon = {fn_city_df['input_ratios'].max():0.3}$"
fig.suptitle(f"{city_suptitle}\n{beta_suptitle}", fontsize=25)

file_name = f"{fx.area}-flatnorm-input-ratios-lambdas_{num_regions}_regions"
close_fig(fig, to_file=f"{fx.fig_dir}/{file_name}.png", show=True)

beta_mean, beta_std

In [ ]:
reg_coefs_data

![](figs/notebooks/mcbryde-flatnorm-input-ratios-lambdas_50_regions.png)

In [ ]:
idx = pd.IndexSlice
y_flatnorms = flatnorm_df.loc[idx[:, lambda_], 'flatnorms'].to_numpy()
x_ratios = flatnorm_df.loc[idx[:, lambda_], 'input_ratios'].to_numpy()
np.corrcoef(x_ratios, y_flatnorms)[0, 1]

# np.polyfit(flatnorm_df['flatnorms'].to_numpy(), flatnorm_df['input_ratios'].to_numpy(), deg=1)

In [ ]:
set("a")

### by lambdas * epsilons

In [ ]:
L = len(lambdas)
E = len(epsilons)
reg_coefs = []
fig, axs = plt.subplots(E, L, figsize=(L*10, E*11), constrained_layout=True)
reg_coefs_dict = dict()
for e, epsilon in enumerate(epsilons):
    for l, lambda_ in enumerate(lambdas):
        reg_coefs_dict[(epsilon, lambda_)] = plot_fn_vs_ratio(
            flatnorm_df, fn_city_df, lambda_, epsilon=epsilon,
            ax=axs[e][l],
            titles=['fn', 'beta', 'r2'],
            title_style="el",
        )

reg_coefs_data = pd.DataFrame(reg_coefs_dict).transpose()
beta_mean, beta_std = reg_coefs_data['b'].mean(), reg_coefs_data['b'].std()

beta_suptitle = f"mean($\\hat{{\\beta}}$)={beta_mean:0.4f}, sd($\\hat{{\\beta}}$)={beta_std:0.4f}"
city_suptitle = f"${{\\bf {fx.area}}} : |T|/\\epsilon = {fn_city_df['input_ratios'].max():0.3}$"
fig.suptitle(f"{city_suptitle}\n{beta_suptitle}", fontsize=25)

file_name = f"{fx.area}-flatnorm-input-ratios_{num_regions}_regions"
close_fig(fig, to_file=f"{fx.fig_dir}/{file_name}.png", show=True)
beta_mean, beta_std


## plot regions and stats

In [ ]:
# a = [1, 2, 3]
a = []
b = "abc"
list(zip(a, b))

### plot_selected_regions()

In [ ]:
def plot_selected_regions(region_ids, fn_df, city_df, lambda_, epsilon=None,
                          axd=dict(), axfn=list(), axtr=list(),
                          **kwargs):
    """
    :param kwargs:
           plot_fn_vs_ratio ::
                    highlight: set, titles: list, title_style: str, X_color: str, X_fontsize: int;
                    if ax is None: figsize, constrained_layout
    """
    kwargs.setdefault('titles', ['fn', 'beta', 'r2'])
    kwargs.setdefault('title_style', "ll")
    colors = {
        # 'city': kwargs.get('city_color', 'xkcd:electric blue'),
        # 'scatter': kwargs.get('scatter_color', 'xkcd:pastel blue'),
        'highlight': kwargs.get('highlight_color', 'xkcd:pumpkin'),
        # 'fn_mean': kwargs.get('fn_mean_color', 'xkcd:kelly green'),
        # 'regression': kwargs.get('regression_color', 'red'),
    }

    if not axd:
        fig, axd = plt.subplot_mosaic(
            [
                ['scatter', 'city'],
            ],
            figsize=kwargs.get('figsize', (12, 6)), constrained_layout=True,
            width_ratios= [ 3, 7],
            gridspec_kw={'wspace': 0.05}
            # gridspec_kw={'width_ratios': [ 3, 7]}
        )
        axfn = list()
        axtr = list()

    stats_data = plot_fn_vs_ratio(
        fn_df, city_df, lambda_, epsilon=epsilon,
        ax=axd['scatter'],
        # flatnorm_df, fn_city_df, lambda_, ax=axs['S'],
        # highlight=fn_df_lambda.loc[fn_df_lambda['flatnorms'] == 1, 'id'].to_list(),
        highlight=region_ids,
        **kwargs
        # titles=['fn', 'beta', 'r2'],
    )

    # read geometries
    act_geom, synt_geom, hull = fx.read_networks()
    selected_regions = dict()
    fn_df = fn_df.set_index('id')
    for r_id in region_ids:
        region_bounds = fn_df.loc[r_id, ['MIN_X', 'MIN_Y', 'MAX_X', 'MAX_Y']].to_list()
        selected_regions[r_id] = sg.box(*region_bounds)

    fx.plot_regions_list(
        act_geom, synt_geom, list(selected_regions.values()), fx.area,
        ax=axd['city'],
        # ax=axs['C'],
        region_color=colors['highlight'],
        # file_name_sfx=f"city",
        do_return=False, show=False,
        **kwargs
        # figsize=(40, 20)
    )

    # plot regions' fn
    if axfn or axtr:
        which_titles = kwargs.get('region_titles', ['epsilon', 'ratio', 'fn'])
        for r, r_id in enumerate(region_ids):
            # flat norm
            # epsilon = fn_df.at[r_id, 'epsilons']
            epsilon, ratio, fnorm = fn_df.loc[r_id, ['epsilons', 'input_ratios', 'flatnorms']]
            region = selected_regions[r_id]
            titles = {
                'lambda': f"$\\lambda = {lambda_:d}$",
                'epsilon': f"$\\epsilon = {epsilon}$",
                'ratio': f"$|T|/\\epsilon = {ratio:0.3g}$",
                'fn': f"$F_{{\\lambda}}={fnorm:0.3g}$",
            }
            title = ", ".join([titles[t_name] for t_name in which_titles])

            D, T1, T2 = fx.get_triangulated_currents(region, act_geom, synt_geom, opts="ps")
            if axtr:
                # tri_title = f"$\\epsilon$ = {epsilon}"
                plot_triangulation(D["triangulated"], T1, T2, axtr[r])
                axtr[r].set_title(title, fontsize=kwargs.get('fontsize', 20))

            if axfn:
                fnorm, enorm, tnorm, w, plot_data = fx.compute_region_flatnorm(
                    D=D, T1=T1, T2=T2,
                    lambda_=lambda_,
                    normalized=True,
                    plot=True
                )
                # fnorm_title =  f"$\\lambda = {lambda_:d}$, $F_{{\\lambda}}={fnorm:0.3g}$"
                plot_norm(plot_data['triangulated'], plot_data['echain'], plot_data['tchain'], axfn[r])
                axfn[r].set_title(title, fontsize=20)


    pass


### F_lambda == 1

In [ ]:
l, lambda_ = 0, lambdas[0]

idx = pd.IndexSlice
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()
fn_df_lambda.sort_values(by='flatnorms', ascending=False)

#### lambda[0] == 1000

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        # ['scatter', 'city'],
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(25, 16), constrained_layout=True,
    width_ratios= [2, 2, 2, 2, 2],
    gridspec_kw={'wspace': 0.05}
)
# )

l, lambda_ = 0, lambdas[0]

idx = pd.IndexSlice
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()
# selected_regions_ids=fn_df_lambda.loc[fn_df_lambda['flatnorms'] == 1, 'id'].to_list()


selected_regions_ids = fn_df_lambda.sort_values(by='flatnorms', ascending=False)['id'].to_list()[:5]


plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5'], ],
)

city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
# fig.suptitle(f"{city_suptitle}\n{beta_suptitle}", fontsize=25)
fig.suptitle(f"{city_suptitle} : max fn", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_max_fn"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


#### lambda[2] == 50500

In [ ]:
np.linspace(1000, 100000, 5, dtype=int)[4]

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        # ['scatter', 'city'],
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(25, 16), constrained_layout=True,
    width_ratios= [2, 2, 2, 2, 2],
    gridspec_kw={'wspace': 0.05}
)
# )

l, lambda_ = 2, lambdas[2]

idx = pd.IndexSlice
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()
# selected_regions_ids=fn_df_lambda.loc[fn_df_lambda['flatnorms'] == 1, 'id'].to_list()


selected_regions_ids = fn_df_lambda.sort_values(by='flatnorms', ascending=False)['id'].to_list()[:5]


plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5'], ],
)

city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
# fig.suptitle(f"{city_suptitle}\n{beta_suptitle}", fontsize=25)
fig.suptitle(f"{city_suptitle} : max fn", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_max_fn"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


#### lambda[4] == 100000

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        # ['scatter', 'city'],
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(25, 16), constrained_layout=True,
    width_ratios= [2, 2, 2, 2, 2],
    gridspec_kw={'wspace': 0.05}
)
# )

l, lambda_ = 4, lambdas[4]

idx = pd.IndexSlice
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()
# selected_regions_ids=fn_df_lambda.loc[fn_df_lambda['flatnorms'] == 1, 'id'].to_list()


selected_regions_ids = fn_df_lambda.sort_values(by='flatnorms', ascending=False)['id'].to_list()[:5]


plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5'], ],
)

city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
# fig.suptitle(f"{city_suptitle}\n{beta_suptitle}", fontsize=25)
fig.suptitle(f"{city_suptitle} : max fn", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_max_fn"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


### closest to city

In [ ]:
city_ratio = fn_city_df['input_ratios'].max()

In [ ]:
fn_df_lambda['to_city'] = abs(fn_df_lambda['input_ratios'] - city_ratio)
fn_df_lambda.sort_values(by='to_city', ascending=True) #['id'].to_list()[:5]

#### lambda[0] == 1000

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(25, 16), constrained_layout=True,
    width_ratios= [2, 2, 2, 2, 2],
    gridspec_kw={'wspace': 0.05}
)

l, lambda_ = 0, lambdas[0]

idx = pd.IndexSlice
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()

city_ratio = fn_city_df['input_ratios'].max()
fn_df_lambda['to_city'] = abs(fn_df_lambda['input_ratios'] - city_ratio)
selected_regions_ids = fn_df_lambda.sort_values(by='to_city', ascending=True)['id'].to_list()[:5]

plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5'], ],
    titles=['fn', 'corr', 'r2'],
)

city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
fig.suptitle(f"{city_suptitle}: close to city", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_close_to_city"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


#### lambda[2] == 50500

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(26, 16), constrained_layout=True,
    gridspec_kw={'width_ratios': [2, 2, 2, 2, 2]}
)

l, lambda_ = 2, lambdas[2]

idx = pd.IndexSlice
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()

city_ratio = fn_city_df['input_ratios'].max()
fn_df_lambda['to_city'] = abs(fn_df_lambda['input_ratios'] - city_ratio)
selected_regions_ids = fn_df_lambda.sort_values(by='to_city', ascending=True)['id'].to_list()[:5]

plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5'], ],
    titles=['fn', 'corr', 'r2'],
)


city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
fig.suptitle(f"{city_suptitle}: close to city", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_close_to_city"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


#### lambda[4] == 100000

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(26, 16), constrained_layout=True,
    width_ratios= [2, 2, 2, 2, 2],
    gridspec_kw={'wspace': 0.05}
)

l, lambda_ = 4, lambdas[4]

idx = pd.IndexSlice
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()

city_ratio = fn_city_df['input_ratios'].max()
fn_df_lambda['to_city'] = abs(fn_df_lambda['input_ratios'] - city_ratio)
selected_regions_ids = fn_df_lambda.sort_values(by='to_city', ascending=True)['id'].to_list()[:5]

plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5'], ],
    titles=['fn', 'corr', 'r2'],
)

city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
fig.suptitle(f"{city_suptitle}: close to city", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_close_to_city"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


### smallest flatnorm

In [ ]:
fn_df_lambda.sort_values(by='to_city', ascending=True)

#### lambda[0] == 1000

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(26, 16), constrained_layout=True,
    width_ratios= [2, 2, 2, 2, 2],
    gridspec_kw={'wspace': 0.05}
)

l, lambda_ = 0, lambdas[0]

idx = pd.IndexSlice
city_ratio = fn_city_df['input_ratios'].max()
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()

selected_regions_ids = fn_df_lambda.sort_values(by='flatnorms', ascending=True)['id'].to_list()[:5]

plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    titles=['fn', 'corr', 'r2'],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5']]
)

city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
fig.suptitle(f"{city_suptitle}: min fn", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_min_fn"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


In [ ]:
fx.fig_dir

#### lambda[2] == 50500

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(26, 16), constrained_layout=True,
    width_ratios= [2, 2, 2, 2, 2],
    gridspec_kw={'wspace': 0.05}
)

l, lambda_ = 2, lambdas[2]

idx = pd.IndexSlice
city_ratio = fn_city_df['input_ratios'].max()
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()

# fn_df_lambda['to_city'] = abs(fn_df_lambda['input_ratios'] - city_ratio)
selected_regions_ids = fn_df_lambda.sort_values(by='flatnorms', ascending=True)['id'].to_list()[:5]

plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5']]
)

# import statistics

# beta_suptitle = f"mean($\\hat{{\\beta}}$)={beta_mean:0.4f}, sd($\\hat{{\\beta}}$)={beta_std:0.4f}"
city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
# fig.suptitle(f"{city_suptitle}\n{beta_suptitle}", fontsize=25)
fig.suptitle(f"{city_suptitle}: min fn", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_min_fn"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


#### lambda[4] == 100000

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)

fig, axd = plt.subplot_mosaic(
    [
        ['scatter', 'scatter', 'city', 'city', 'city'],
        ['t1', 't2', 't3', 't4', 't5'],
        # ['r1', 'r2', 'r3', 'r4', 'r5'],
    ],
    figsize=(26, 16), constrained_layout=True,
    width_ratios= [2, 2, 2, 2, 2],
    gridspec_kw={'wspace': 0.05}
)

l, lambda_ = 4, lambdas[4]

idx = pd.IndexSlice
city_ratio = fn_city_df['input_ratios'].max()
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()

selected_regions_ids = fn_df_lambda.sort_values(by='flatnorms', ascending=True)['id'].to_list()[:5]

plot_selected_regions(
    selected_regions_ids, flatnorm_df, fn_city_df, lambda_,
    axd=axd,
    axtr=[axd['t1'], axd['t2'], axd['t3'], axd['t4'], axd['t5']],
    titles=['fn', 'corr', 'r2'],
    # axfn=[axd['r1'], axd['r2'], axd['r3'], axd['r4'], axd['r5']]
)

city_suptitle = f"$\\it{{{fx.area}}} : |T|/\\epsilon = {city_ratio:0.3}$"
fig.suptitle(f"{city_suptitle}: min fn", fontsize=25)
file_name = f"{fx.area}-selected_regions_l{lambda_}_min_fn"
close_fig(fig, to_file=f"{fx.fig_dir}/regions/{file_name}.png", show=True)


### below reg line

In [ ]:
reg_coefs_data

In [ ]:
lambdas = np.linspace(1000, 100000, 5, dtype=int)
L = len(lambdas)
l, lambda_ = 4, lambdas[4]

idx = pd.IndexSlice
city_ratio = fn_city_df['input_ratios'].max()
fn_df_lambda = flatnorm_df.loc[idx[:, lambda_], ].copy()
fn_df_lambda

In [ ]:
a, b = reg_coefs_data.loc[lambda_, ['a', 'b']]
fn_df_lambda['fn_hat'] = a + b * fn_df_lambda['input_ratios']
fn_df_lambda['fn_hat_0'] =fn_df_lambda['flatnorms'] -  fn_df_lambda['fn_hat']
fn_df_lambda

#### lambda[0] == 1000

#### lambda[2] == 50500

#### lambda[4] == 100000


# Hethwood

In [ ]:
fx_heth = FlatNormFixture('runTest')
fx_heth.fig_dir = "figs/notebooks"
fx_heth.out_dir = "outs/notebooks"
fx_heth.area = 'hethwood'


In [ ]:
# read geometries
mc_act_geom, mc_synt_geom, mc_hull = fx.read_networks(fx.area)
heth_act_geom, heth_synt_geom, heth_hull = fx_heth.read_networks(fx_heth.area)

mc_bounds = mc_hull.exterior.bounds
mc_width, mc_height = mc_bounds[MAX_X] - mc_bounds[MIN_X], mc_bounds[MAX_Y] - mc_bounds[MIN_Y]
mc_epsilon = max(mc_width/2, mc_height/2)

heth_bounds = heth_hull.exterior.bounds
heth_width, heth_height = heth_bounds[MAX_X] - heth_bounds[MIN_X], heth_bounds[MAX_Y] - heth_bounds[MIN_Y]
heth_epsilon = max(heth_width/2, heth_height/2)

mc_epsilon, heth_epsilon

In [ ]:
mc_epsilon

In [ ]:
len(mc_act_geom), len(mc_synt_geom), len(heth_act_geom), len(heth_synt_geom)


In [ ]:
k = np.pi / 180.0
mc_act_length = [g.length * k for g in mc_act_geom]
mc_synt_length = [g.length * k for g in mc_synt_geom]
sum(mc_act_length), sum(mc_synt_length), sum(mc_act_length) + sum(mc_synt_length)

In [ ]:
k = np.pi / 180.0
heth_act_length = [g.length * k for g in heth_act_geom]
heth_synt_length = [g.length * k for g in heth_synt_geom]
sum(heth_act_length), sum(heth_synt_length), sum(heth_act_length) + sum(heth_synt_length)

In [ ]:
mc_act_geom[0].length


In [ ]:
sg.box(*mc_hull.exterior.bounds).bounds

In [ ]:
# mc_current_length = w
mc_current_length

In [ ]:
mc_current_length - (sum(mc_act_length) + sum(mc_synt_length))


In [ ]:
min(mc_act_length + mc_synt_length), max(mc_act_length + mc_synt_length),

In [ ]:
min(heth_act_length + heth_synt_length), max(heth_act_length + heth_synt_length),


In [ ]:
len(heth_act_length + heth_synt_length), len(heth_act_length) + len(heth_synt_length),
len(mc_act_length + mc_synt_length), len(mc_act_length) + len(mc_synt_length),

In [ ]:
(
        min(heth_act_length + heth_synt_length) < min(mc_act_length + mc_synt_length),
        max(heth_act_length + heth_synt_length) < max(mc_act_length + mc_synt_length)
)


In [ ]:
min(heth_act_length + heth_synt_length) < min(mc_act_length + mc_synt_length)


In [ ]:
D, T1, T2 = fx_heth.get_triangulated_currents(sg.box(*heth_bounds), heth_act_geom, heth_synt_geom, verbose=True)
len(T1)
